<a href="https://colab.research.google.com/github/anismenaa/chainage/blob/main/Colab_Esi_sba_Ex2_AVG_HousePrice_By_City.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Installation de spark-cluster & findspark, pyspark 

In [1]:
# innstall java
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

# install spark (change the version number if needed)
!wget -q https://dlcdn.apache.org/spark/spark-3.2.0/spark-3.2.0-bin-hadoop2.7.tgz

# unzip the spark file to the current folder
!tar xf spark-3.2.0-bin-hadoop2.7.tgz

# set your spark folder to your system path environment. 
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.2.0-bin-hadoop2.7"


# install findspark using pip
!pip install -q findspark

# install pyspark
!pip install pyspark

     |████████████████████████████████| 281.3 MB 39 kB/s 
     |████████████████████████████████| 198 kB 56.7 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.0-py2.py3-none-any.whl size=281805912 sha256=6dee7f24af93d29ad65e9e9e48ba773e656413963c105c38da56b7142b63d38e
  Stored in directory: /root/.cache/pip/wheels/0b/de/d2/9be5d59d7331c6c2a7c1b6d1a4f463ce107332b1ecd4e80718
Successfully built pyspark


In [2]:
import findspark 
findspark.init()
from pyspark import SparkContext, SparkConf
conf = SparkConf().setAppName("actions").setMaster("local[4]")
sc = SparkContext(conf = conf)

In [4]:
#Charger le dataset HousePrice.csv

lines = sc.textFile("HousePrice.csv")
#nettoyer le dataset en supprimant ceux qui n'ont pas une city ou un price
cleanedLines = lines.filter(lambda line : "city" not in line or "price" not in line)
cleanedLines.take(2)



['2014-05-02 00:00:00,313000.0,3.0,1.5,1340,7912,1.5,0,0,3,1340,0,1955,2005,18810 Densmore Ave N,Shoreline,WA 98133,USA',
 '2014-05-02 00:00:00,2384000.0,5.0,2.5,3650,9050,2.0,0,4,5,3370,280,1921,0,709 W Blaine St,Seattle,WA 98119,USA']

In [7]:
# récupérer le price(position 1) et la city (position 15)

City_Price_Rdd=cleanedLines.map(lambda line : (line.split(",")[15], float(line.split(",")[1])))
City_HouseNB_Rdd = cleanedLines.map(lambda line : (line.split(",")[15], 1))

In [8]:
# calculer le prix moyen pour chaque city
city_Price_Total_Rdd=City_Price_Rdd.reduceByKey(lambda x,y : x+y)
City_HouseNB_Total_Rdd = City_HouseNB_Rdd.reduceByKey(lambda x,y : x+y)
AVG = city_Price_Total_Rdd.union(City_HouseNB_Total_Rdd).reduceByKey(lambda x,y :x/y)
AVG.collect()

[('Seattle', 579837.4672823235),
 ('Redmond', 667649.5347812298),
 ('Maple Valley', 336474.9118055521),
 ('North Bend', 399565.70666666003),
 ('Sammamish', 686917.5835323657),
 ('Auburn', 299340.44276556815),
 ('Bothell', 481441.8939393939),
 ('Federal Way', 289887.70398969593),
 ('Kirkland', 651583.5919080963),
 ('Woodinville', 609565.0217391305),
 ('Normandy Park', 506793.05555555556),
 ('Renton', 377040.96632902045),
 ('Duvall', 403994.12698411907),
 ('Kenmore', 447494.0404040303),
 ('Mercer Island', 1123817.6201550348),
 ('Black Diamond', 339605.55555555556),
 ('Ravensdale', 514071.4285714286),
 ('Algona', 207288.0),
 ('Tukwila', 308290.0689655172),
 ('Vashon', 472556.8965517241),
 ('Yarrow Point', 1194837.5),
 ('SeaTac', 245290.62068965516),
 ('Medina', 2046559.0909090908),
 ('Enumclaw', 307614.58333332144),
 ('Pacific', 225233.33333333334),
 ('Preston', 562450.0),
 ('Shoreline', 420392.3640469756),
 ('Kent', 439492.44464821625),
 ('Bellevue', 847180.6629953358),
 ('Lake Forest Pa